### Задание

Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision ([datasets](https://pytorch.org/vision/stable/datasets.html)).

Получите качество на тестовой выборке не ниже 88%

Инструкция по выполнению задания:
    Скачайте тренировочную и тестовою часть датасета Fashion MNIST
    Постройте модель, выбрав стартовую архитектуру
    Обучите модель и сверьте качество на тестовой части с заданным порогом
    Изменяйте архитектуру модели пока качество на тестовой части не будет выше порога. Вариации архитектуры можно реализовать через изменение количества слоёв, количества нейронов в слоях и использование регуляризации. Можно использовать различные оптимизаторы.


In [1]:
# импорт библиотек
import torch # импортирую основной торч
# импортирую библиотеку torchvision для машинного обучения, которая содержит датасеты
    # import torchvision as tv # можно так, через карткую запись с "tv"
import torchvision # буду использовать полную запись
import time

In [2]:
# подготовка
BATCH_SIZE=256 # указываю размер batch

In [3]:
# загружаю и подготавливаю тренировочный и тестовы датасеты

train_dataset = torchvision.datasets.FashionMNIST(
    '.', # указание на директорию, куда скачивать
    train=True, # указание на тренировочный датасет "If True, creates dataset from train-images-idx3-ubyte, otherwise from t10k-images-idx3-ubyte"
    transform=torchvision.transforms.ToTensor(), # преобразование изображений в тензоры
    download=True # скачивать надо
    )
test_dataset = torchvision.datasets.FashionMNIST('.', train=False, transform=torchvision.transforms.ToTensor(), download=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE) # тренировочный датасет
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE) # тестовы датасет

In [4]:
def train_model(num_epochs=10): # функция обучения модели. её скопировал из лекции, но добавил количество эпох в аргументы
    for ep in range(num_epochs): # итерируемся по количеству эпох
        train_iters, train_passed  = 0, 0 # для каждой эпохи задаём характеристики обучения, куда будут записываться результаты
        train_loss, train_acc = 0., 0.
        start=time.time() # фиксируем время начала обучения
        
        # тренировочный модуль
        model.train() # переводим модель в тренировочный режим
        for X, y in train: # итерируемся по тренировочной части датасета
            trainer.zero_grad() # сбрасываем градиенты
            y_pred = model(X) # считаем предсказания модели
            l = loss(y_pred, y) # считаем функцию потерь (y - позиция правильного ответа в векторе)
            l.backward() # считаем градиенты
            trainer.step() # делаем шаг градиентного спуска
            train_loss += l.item() # фиксируем количество элементов, которое пропустили через модель
            train_acc += (y_pred.argmax(dim=1) == y).sum().item() # фиксируем метрику accuracy. сравниваем позицию максимума в каждом ответе с правильным ответом.
                                                                  # суммируем и переводим в скаляр.
            train_iters += 1 # количество итераций
            train_passed += len(X) # сколько всего данных пропустили через модель
        
        # тестовый модуль. всё то же самое, кроме сброса и рассчёта градиентов
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)

        # печатаем результаты    
        print("ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

##### Попытка 1. Использую самые простые параметры на основании материалов лекции.
Захожу с козырей - беру саму эффективную модель и оптимизатор "adam" с достаточно маленьким lr.

In [22]:
# модель с 3 скрытыми слоями и батч-нормализацией перед функцией активации. в экспериментах на материалах лекции показала самые высокие результаты
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),    
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 128),
    torch.nn.BatchNorm1d(128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)

In [23]:
# настроки функции потерь, оптимизатора и запуск модели
loss = torch.nn.CrossEntropyLoss() # функция потерь
trainer = torch.optim.Adam(model.parameters(), lr=.001)
train_model(10)

ep: 0, taked: 6.204, train_loss: 0.460247773058871, train_acc: 0.84305, test_loss: 0.3834825411438942, test_acc: 0.8618
ep: 1, taked: 6.235, train_loss: 0.3177239336231922, train_acc: 0.8829833333333333, test_loss: 0.36832467671483754, test_acc: 0.8654
ep: 2, taked: 6.226, train_loss: 0.27560252781878125, train_acc: 0.89835, test_loss: 0.35442294646054506, test_acc: 0.8691
ep: 3, taked: 6.096, train_loss: 0.24404084213236546, train_acc: 0.90975, test_loss: 0.3405782756395638, test_acc: 0.8752
ep: 4, taked: 6.205, train_loss: 0.21931247267317264, train_acc: 0.9180333333333334, test_loss: 0.35044979508966206, test_acc: 0.8731
ep: 5, taked: 6.169, train_loss: 0.20014272367066524, train_acc: 0.9263333333333333, test_loss: 0.3502777867950499, test_acc: 0.8756
ep: 6, taked: 6.110, train_loss: 0.1819811965873901, train_acc: 0.93375, test_loss: 0.34277742379345, test_acc: 0.8812
ep: 7, taked: 6.016, train_loss: 0.16807523414175563, train_acc: 0.9381166666666667, test_loss: 0.36763629904016853,

##### Результат не очень. В какой-то момент accuracy на тестовых данных достигает заданного показателля в 88%, но функция потерь растёт и налицо явное переобучение. Нужно продолжать тренироваться дальше.

##### Попытка 2. То же, но меняю оптимизатор на SGD.

In [24]:
loss = torch.nn.CrossEntropyLoss() # функция потерь
trainer = torch.optim.SGD(model.parameters(), lr=.001)
train_model(10)

ep: 0, taked: 5.830, train_loss: 0.14431609719040547, train_acc: 0.9469666666666666, test_loss: 0.3550504838640336, test_acc: 0.8917
ep: 1, taked: 5.778, train_loss: 0.13667895583079218, train_acc: 0.9501166666666667, test_loss: 0.34898901914712044, test_acc: 0.893
ep: 2, taked: 5.898, train_loss: 0.13190667549980448, train_acc: 0.9517833333333333, test_loss: 0.3449986718886066, test_acc: 0.8935
ep: 3, taked: 5.756, train_loss: 0.12842598116778312, train_acc: 0.9532333333333334, test_loss: 0.34207663861452603, test_acc: 0.895
ep: 4, taked: 5.781, train_loss: 0.12567516430578332, train_acc: 0.9546666666666667, test_loss: 0.33978704465553167, test_acc: 0.8948
ep: 5, taked: 5.850, train_loss: 0.12339214991698874, train_acc: 0.9557, test_loss: 0.33793656177585946, test_acc: 0.8952
ep: 6, taked: 5.878, train_loss: 0.12143496175078636, train_acc: 0.95655, test_loss: 0.33638957106741146, test_acc: 0.8959
ep: 7, taked: 5.799, train_loss: 0.11972269322326842, train_acc: 0.9571, test_loss: 0.335

##### Уже гораздо лучше. Accuracy стабильно выше 88%, функция потерь медленно, но верно продолжает падать.
##### В принципе, задание выполнено.

##### Попытка 3. Расширяю слои и добавляю dropout = 0.3

In [29]:
# получилось чудовище, которое очень долго обучается, поэтому часть слоёв убрал - ниже закомментировано.
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    # torch.nn.Linear(784, 21952),
    # torch.nn.BatchNorm1d(21952),
    # torch.nn.ReLU(),
    # torch.nn.Dropout(0.3),     
    # torch.nn.Linear(21952, 10976),
    # torch.nn.BatchNorm1d(10976),
    # torch.nn.ReLU(),
    # torch.nn.Dropout(0.3), 
    # torch.nn.Linear(10976, 5488),
    torch.nn.Linear(784, 5488), # решил попробовать стартовать с этого слоя
    torch.nn.BatchNorm1d(5488),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(5488, 2744),
    torch.nn.BatchNorm1d(2744),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(2744, 1372),
    torch.nn.BatchNorm1d(1372),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(1372, 686),
    torch.nn.BatchNorm1d(686),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(686, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(256, 128),
    torch.nn.BatchNorm1d(128),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3), 
    torch.nn.Linear(128, 10)
)

In [30]:
trainer = torch.optim.Adam(model.parameters(), lr=.001)
train_model(10)

ep: 0, taked: 43.834, train_loss: 0.5677300082876328, train_acc: 0.8124, test_loss: 0.44351678937673567, test_acc: 0.8397
ep: 1, taked: 40.640, train_loss: 0.4004015198413362, train_acc: 0.85865, test_loss: 0.40568487998098135, test_acc: 0.8511
ep: 2, taked: 40.148, train_loss: 0.3575161997308122, train_acc: 0.8727833333333334, test_loss: 0.42818290926516056, test_acc: 0.8386
ep: 3, taked: 40.087, train_loss: 0.33379341205383867, train_acc: 0.8815, test_loss: 0.36959649808704853, test_acc: 0.8622
ep: 4, taked: 41.911, train_loss: 0.3121816133565091, train_acc: 0.8879833333333333, test_loss: 0.3600862676277757, test_acc: 0.8676
ep: 5, taked: 42.552, train_loss: 0.3004648819882819, train_acc: 0.89235, test_loss: 0.36051029302179816, test_acc: 0.8652
ep: 6, taked: 42.705, train_loss: 0.2832642280674995, train_acc: 0.89825, test_loss: 0.3502289186231792, test_acc: 0.8751
ep: 7, taked: 42.108, train_loss: 0.27086814255156416, train_acc: 0.90345, test_loss: 0.3411148852668703, test_acc: 0.87

##### Получилось не очень. Целевая accuracy не достигнута.

##### Попытка 4. То же, но снова меняю оптимизатор на SGD.

In [31]:
trainer = torch.optim.SGD(model.parameters(), lr=.001)
train_model(10)

ep: 0, taked: 32.762, train_loss: 0.23326570600905316, train_acc: 0.9148333333333334, test_loss: 0.3131233058869839, test_acc: 0.8891
ep: 1, taked: 32.380, train_loss: 0.22981907566811177, train_acc: 0.9163166666666667, test_loss: 0.3105848267674446, test_acc: 0.8904
ep: 2, taked: 31.906, train_loss: 0.2304696095116595, train_acc: 0.9175, test_loss: 0.30987528720870616, test_acc: 0.8902
ep: 3, taked: 30.961, train_loss: 0.2281319041835501, train_acc: 0.9179, test_loss: 0.30962988631799815, test_acc: 0.8907
ep: 4, taked: 30.212, train_loss: 0.22473770659020606, train_acc: 0.9195833333333333, test_loss: 0.30886369831860067, test_acc: 0.8903
ep: 5, taked: 30.989, train_loss: 0.22758651806953106, train_acc: 0.9191, test_loss: 0.3086301933974028, test_acc: 0.8903
ep: 6, taked: 30.132, train_loss: 0.22599609722482397, train_acc: 0.9186, test_loss: 0.3083180967718363, test_acc: 0.8906
ep: 7, taked: 30.280, train_loss: 0.22253187127569887, train_acc: 0.9200333333333334, test_loss: 0.3081001692

##### Вообще красота. Accuracy стабильно больше 89%. Значение функции потерь медленно снижается.
##### Ниже ставлю эксперимент на 50 эпохах. Эффект примерно такой же функция потерь и accuracy стабилизируются.

In [32]:
trainer = torch.optim.SGD(model.parameters(), lr=.001)
train_model(50)

ep: 0, taked: 30.268, train_loss: 0.22299631396506694, train_acc: 0.9202166666666667, test_loss: 0.30767538454383614, test_acc: 0.8909
ep: 1, taked: 30.469, train_loss: 0.22266264976339137, train_acc: 0.92075, test_loss: 0.30765434904024, test_acc: 0.8906
ep: 2, taked: 30.201, train_loss: 0.2238341744592849, train_acc: 0.9205666666666666, test_loss: 0.3076131368987262, test_acc: 0.8907
ep: 3, taked: 29.956, train_loss: 0.2231611206810525, train_acc: 0.91965, test_loss: 0.3073070764541626, test_acc: 0.8908
ep: 4, taked: 30.141, train_loss: 0.2213785765970007, train_acc: 0.9203833333333333, test_loss: 0.3074204076081514, test_acc: 0.8908
ep: 5, taked: 29.998, train_loss: 0.2219564601461938, train_acc: 0.9204833333333333, test_loss: 0.3072748835198581, test_acc: 0.8907
ep: 6, taked: 30.493, train_loss: 0.22182581722736358, train_acc: 0.9202666666666667, test_loss: 0.3069695131853223, test_acc: 0.8912
ep: 7, taked: 30.069, train_loss: 0.22125112667996832, train_acc: 0.9210833333333334, tes